In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/optiver-realized-volatility-prediction/sample_submission.csv
/kaggle/input/optiver-realized-volatility-prediction/train.csv
/kaggle/input/optiver-realized-volatility-prediction/test.csv
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=97/888f813404d8417ca8d6b8aebd5f2951.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=43/bb0efa57f511470e817880842e3e2afa.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=21/1d8dc18ebfee47ffbb54b04e6afc0634.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=72/60f62a03d8854605901dda072c84db39.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=4/761268d671f9429abb29d9d2895e9bd2.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=112/cd283097a5b54293ba400a19e811a7f9.parquet
/kaggle/input/optiver-realized-volatility-pr

In [2]:
import os
import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
from sklearn.model_selection import KFold
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)
from sklearn.model_selection import GroupKFold

In [3]:
# data directory
data_dir = '../input/optiver-realized-volatility-prediction/'

# Function to calculate first WAP
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# Function to calculate second WAP
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# Function to calculate the log of the return
# Remember that logb(x / y) = logb(x) - logb(y)
def log_return(series):
    return np.log(series).diff()

# Calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

In [4]:
# Function to read our base train and test set
def read_train_test():
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    # Create a key to merge with book and trade data
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    return test

In [5]:
# Function to preprocess book data (for each stock id)
def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    # Calculate Wap
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    # Calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    
    df['wap_mean'] = (df['wap1'] + df['wap2']) / 2
    # Calculate wap balance
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    df['bas'] = (df[['ask_price1', 'ask_price2']].min(axis = 1)/ df[['bid_price1', 'bid_price2']].max(axis = 1) - 1) 
    df['h_spread_l1'] = df['ask_price1'] - df['bid_price1']
    df['h_spread_l2'] = df['ask_price2'] - df['bid_price2']
    df['log_return_bid_price1'] = np.log(df['bid_price1'].pct_change() + 1)
    df['log_return_ask_price1'] = np.log(df['ask_price1'].pct_change() + 1)
    df['log_return_bid_size1'] = np.log(df['bid_size1'].pct_change() + 1)
    df['log_return_ask_size1'] = np.log(df['ask_size1'].pct_change() + 1)
    df['log_ask_1_div_bid_1'] = np.log(df['ask_price1'] / df['bid_price1'])
    df['log_ask_1_div_bid_1_size'] = np.log(df['ask_size1'] / df['bid_size1'])
    df['log_return_bid_price2'] = np.log(df['bid_price2'].pct_change() + 1)
    df['log_return_ask_price2'] = np.log(df['ask_price2'].pct_change() + 1)
    df['log_return_bid_size2'] = np.log(df['bid_size2'].pct_change() + 1)
    df['log_return_ask_size2'] = np.log(df['ask_size2'].pct_change() + 1)
    df['log_ask_2_div_bid_2'] = np.log(df['ask_price2'] / df['bid_price2'])
    df['log_ask_2_div_bid_2_size'] = np.log(df['ask_size2'] / df['bid_size2'])
    
    
    # Dict for aggregations
    create_feature_dict = {
        # 测试1
        'wap1': [np.mean,np.max,np.std],#np.sum, 上次测试的结果
        'wap2': [np.mean,np.max,np.std],#np.sum, 上次测试的结果
        # np.mean, np.max 加入 sum
        #测试2
        'log_return1': [realized_volatility,np.sum], # np.sum, np.mean,np.std
        'log_return2': [realized_volatility,np.sum], # np.sum, np.mean,np.std
        'log_return_bid_price1':[realized_volatility,np.sum], # np.sum, np.mean,np.std
        'log_return_ask_price1':[realized_volatility,np.sum], # np.sum, np.mean,np.std
        'log_return_bid_size1':[realized_volatility,np.sum], # np.sum, np.mean,np.std
        'log_return_ask_size1':[realized_volatility,np.sum], # np.sum, np.mean,np.std
        'log_ask_1_div_bid_1':[realized_volatility,np.sum], # np.sum, np.mean,np.std
        'log_ask_1_div_bid_1_size':[realized_volatility,np.sum], # np.sum, np.mean,np.std
        'log_return_bid_price2':[realized_volatility,np.sum], # np.sum, np.mean,np.std
        'log_return_ask_price2':[realized_volatility,np.sum], # np.sum, np.mean,np.std
        'log_return_bid_size2':[realized_volatility,np.sum], # np.sum, np.mean,np.std
        'log_return_ask_size2':[realized_volatility,np.sum], # np.sum, np.mean,np.std
        'log_ask_2_div_bid_2':[realized_volatility,np.sum], # np.sum, np.mean,np.std
        'log_ask_2_div_bid_2_size':[realized_volatility,np.sum], # np.sum, np.mean,np.std
        #测试3
        'wap_balance': [np.mean,np.max,np.std,np.sum], #上次测试的结果, np.sum
        'wap_mean': [np.mean,np.max,np.std,np.sum],#上次测试的结果, np.sum
        'price_spread':[np.mean,np.max,np.std,np.sum],#上次测试的结果, np.sum
        'bid_spread':[np.mean,np.max,np.std,np.sum],#上次测试的结果, np.sum
        'ask_spread':[np.mean,np.max,np.std,np.sum],#上次测试的结果, np.sum
        'total_volume':[np.mean,np.max,np.std,np.sum],#上次测试的结果, np.sum
        'volume_imbalance':[np.mean,np.max,np.std,np.sum],#上次测试的结果, np.sum
        'bas':[np.mean,np.max,np.std,np.sum],#上次测试的结果, np.sum
        'h_spread_l1':[np.mean,np.max,np.std,np.sum],#上次测试的结果, np.sum
        'h_spread_l2':[np.mean,np.max,np.std,np.sum],#上次测试的结果, np.sum
         #np.mean, np.max 加入 sum
                       }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_450 = get_stats_window(seconds_in_bucket = 450, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_150 = get_stats_window(seconds_in_bucket = 150, add_suffix = True)
    
    # Merge all
    df_feature = df_feature.merge(df_feature_450, how = 'left', left_on = 'time_id_', right_on = 'time_id__450')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_150, how = 'left', left_on = 'time_id_', right_on = 'time_id__150')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__450', 'time_id__300', 'time_id__150'], axis = 1, inplace = True)
    
    # Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature

In [6]:
# Function to preprocess trade data (for each stock id)
def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    # Dict for aggregations
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.mean],
    }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_feature_450 = get_stats_window(seconds_in_bucket = 450, add_suffix = True)
    df_feature_300 = get_stats_window(seconds_in_bucket = 300, add_suffix = True)
    df_feature_150 = get_stats_window(seconds_in_bucket = 150, add_suffix = True)

    # Merge all
    df_feature = df_feature.merge(df_feature_450, how = 'left', left_on = 'time_id_', right_on = 'time_id__450')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_150, how = 'left', left_on = 'time_id_', right_on = 'time_id__150')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__450', 'time_id__300', 'time_id__150'], axis = 1, inplace = True)
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature


In [7]:
# Function to get group stats for the stock_id and time_id
def get_time_stock(df):
    # Get realized volatility columns
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_450', 'log_return2_realized_volatility_450', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_150', 'log_return2_realized_volatility_150', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_450', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_150']

    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the stock id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df

In [8]:
# Funtion to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, is_train = True):
    
    # Parrallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        # Preprocess book and trade data and merge them
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        
        # Return the merge dataframe
        return df_tmp
    
    # Use parallel api to call paralle for loop
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(df, ignore_index = True)
    return df

In [9]:
# Read train and test
test = read_train_test()


# Get unique stock ids 
test_stock_ids = test['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')

# Get group stats of time_id and stock_id
test = get_time_stock(test)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.3s finished


In [10]:
train = pd.read_pickle('../input/train-data/train.pkl') #/kaggle/input/train-data/train.pkl

In [11]:
from sklearn import datasets
from sklearn import model_selection

def create_folds(data, num_splits,target):
    # we create a new column called kfold and fill it with -1
    data["kfold"] = -1
    
    # the next step is to randomize the rows of the data
    data = data.sample(frac=1).reset_index(drop=True)

    # calculate number of bins by Sturge's rule
    # I take the floor of the value, you can also
    # just round it
    num_bins = int(np.floor(1 + np.log2(len(data))))
    
    # bin targets
    data.loc[:, "bins"] = pd.cut(
        data[target], bins=num_bins, labels=False
    )
    
    # initiate the kfold class from model_selection module
    kf = model_selection.StratifiedKFold(n_splits=num_splits)
    
    # fill the new kfold column
    # note that, instead of targets, we use bins!
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'kfold'] = f
    
    # drop the bins column
    data = data.drop("bins", axis=1)

    # return dataframe with folds
    return data

train = create_folds(train, 10,"target")

In [12]:
train.head()

,stock_id,time_id,target,row_id,wap1_mean,wap1_amax,wap1_std,wap2_mean,wap2_amax,wap2_std,log_return1_realized_volatility,log_return1_sum,log_return2_realized_volatility,log_return2_sum,log_return_bid_price1_realized_volatility,log_return_bid_price1_sum,log_return_ask_price1_realized_volatility,log_return_ask_price1_sum,log_return_bid_size1_realized_volatility,log_return_bid_size1_sum,log_return_ask_size1_realized_volatility,log_return_ask_size1_sum,log_ask_1_div_bid_1_realized_volatility,log_ask_1_div_bid_1_sum,log_ask_1_div_bid_1_size_realized_volatility,log_ask_1_div_bid_1_size_sum,log_return_bid_price2_realized_volatility,log_return_bid_price2_sum,log_return_ask_price2_realized_volatility,log_return_ask_price2_sum,log_return_bid_size2_realized_volatility,log_return_bid_size2_sum,log_return_ask_size2_realized_volatility,log_return_ask_size2_sum,log_ask_2_div_bid_2_realized_volatility,log_ask_2_div_bid_2_sum,log_ask_2_div_bid_2_size_realized_volatility,log_ask_2_div_bid_2_size_sum,wap_balance_mean,wap_balance_amax,wap_balance_std,wap_balance_sum,wap_mean_mean,wap_mean_amax,wap_mean_std,wap_mean_sum,price_spread_mean,price_spread_amax,price_spread_std,price_spread_sum,bid_spread_mean,bid_spread_amax,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_amax,ask_spread_std,ask_spread_sum,total_volume_mean,total_volume_amax,total_volume_std,total_volume_sum,volume_imbalance_mean,volume_imbalance_amax,volume_imbalance_std,volume_imbalance_sum,bas_mean,bas_amax,bas_std,bas_sum,h_spread_l1_mean,h_spread_l1_amax,h_spread_l1_std,h_spread_l1_sum,h_spread_l2_mean,h_spread_l2_amax,h_spread_l2_std,h_spread_l2_sum,wap1_mean_450,wap1_amax_450,wap1_std_450,wap2_mean_450,wap2_amax_450,wap2_std_450,log_return1_realized_volatility_450,log_return1_sum_450,log_return2_realized_volatility_450,log_return2_sum_450,log_return_bid_price1_realized_volatility_450,log_return_bid_price1_sum_450,log_return_ask_price1_realized_volatility_450,log_return_ask_price1_sum_450,log_return_bid_size1_realized_volatility_450,log_return_bid_size1_sum_450,log_return_ask_size1_realized_volatility_450,log_return_ask_size1_sum_450,log_ask_1_div_bid_1_realized_volatility_450,log_ask_1_div_bid_1_sum_450,log_ask_1_div_bid_1_size_realized_volatility_450,log_ask_1_div_bid_1_size_sum_450,log_return_bid_price2_realized_volatility_450,log_return_bid_price2_sum_450,log_return_ask_price2_realized_volatility_450,log_return_ask_price2_sum_450,log_return_bid_size2_realized_volatility_450,log_return_bid_size2_sum_450,log_return_ask_size2_realized_volatility_450,log_return_ask_size2_sum_450,log_ask_2_div_bid_2_realized_volatility_450,log_ask_2_div_bid_2_sum_450,log_ask_2_div_bid_2_size_realized_volatility_450,log_ask_2_div_bid_2_size_sum_450,wap_balance_mean_450,wap_balance_amax_450,wap_balance_std_450,wap_balance_sum_450,wap_mean_mean_450,wap_mean_amax_450,wap_mean_std_450,wap_mean_sum_450,price_spread_mean_450,price_spread_amax_450,price_spread_std_450,price_spread_sum_450,bid_spread_mean_450,bid_spread_amax_450,bid_spread_std_450,bid_spread_sum_450,ask_spread_mean_450,ask_spread_amax_450,ask_spread_std_450,ask_spread_sum_450,total_volume_mean_450,total_volume_amax_450,total_volume_std_450,total_volume_sum_450,volume_imbalance_mean_450,volume_imbalance_amax_450,volume_imbalance_std_450,volume_imbalance_sum_450,bas_mean_450,bas_amax_450,bas_std_450,bas_sum_450,h_spread_l1_mean_450,h_spread_l1_amax_450,h_spread_l1_std_450,h_spread_l1_sum_450,h_spread_l2_mean_450,h_spread_l2_amax_450,...,wap_balance_sum_150,wap_mean_mean_150,wap_mean_amax_150,wap_mean_std_150,wap_mean_sum_150,price_spread_mean_150,price_spread_amax_150,price_spread_std_150,price_spread_sum_150,bid_spread_mean_150,bid_spread_amax_150,bid_spread_std_150,bid_spread_sum_150,ask_spread_mean_150,ask_spread_amax_150,ask_spread_std_150,ask_spread_sum_150,total_volume_mean_150,total_volume_amax_150,total_volume_std_150,total_volume_sum_150,volume_imbalance_mean_150,volume_imbalance_amax_150,volume_imbalance_std_150,volume_im

In [13]:
# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False


In [14]:
# Split features and target
x = train.drop(['row_id', 'target', 'time_id'], axis = 1)
y = train['target']
x_test = test.drop(['row_id', 'time_id'], axis = 1)
    # Transform stock id to a numeric value
x['stock_id'] = x['stock_id'].astype(int)
x_test['stock_id'] = x_test['stock_id'].astype(int)
    
 # Create out of folds array
oof_predictions = np.zeros(x.shape[0])
# Create test array to store predictions
test_predictions = np.zeros(x_test.shape[0])
# # Create a groupFold object
# groupkfold = GroupKFold(n_splits = 10, shuffle=True, random_state=42)
# # Iterate through each fold
# for fold, (trn_ind, val_ind) in enumerate(groupkfold.split(train,train['target'],'time_id')):
#     x_train, x_val = x.iloc[trn_ind], x.iloc[val_ind]
#     y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
for fold in range(10):
        traindf = train[train["kfold"]!=fold].reset_index(drop=True)
        validdf = train[train["kfold"]==fold].reset_index(drop=True)
        x_train = traindf.drop(['row_id', 'target', 'time_id',"kfold"], axis = 1)
        y_train = traindf['target']
        x_val = validdf.drop(['row_id', 'target', 'time_id',"kfold"], axis = 1)
        y_val =  validdf['target']

In [15]:
x_train.head()

,stock_id,wap1_mean,wap1_amax,wap1_std,wap2_mean,wap2_amax,wap2_std,log_return1_realized_volatility,log_return1_sum,log_return2_realized_volatility,log_return2_sum,log_return_bid_price1_realized_volatility,log_return_bid_price1_sum,log_return_ask_price1_realized_volatility,log_return_ask_price1_sum,log_return_bid_size1_realized_volatility,log_return_bid_size1_sum,log_return_ask_size1_realized_volatility,log_return_ask_size1_sum,log_ask_1_div_bid_1_realized_volatility,log_ask_1_div_bid_1_sum,log_ask_1_div_bid_1_size_realized_volatility,log_ask_1_div_bid_1_size_sum,log_return_bid_price2_realized_volatility,log_return_bid_price2_sum,log_return_ask_price2_realized_volatility,log_return_ask_price2_sum,log_return_bid_size2_realized_volatility,log_return_bid_size2_sum,log_return_ask_size2_realized_volatility,log_return_ask_size2_sum,log_ask_2_div_bid_2_realized_volatility,log_ask_2_div_bid_2_sum,log_ask_2_div_bid_2_size_realized_volatility,log_ask_2_div_bid_2_size_sum,wap_balance_mean,wap_balance_amax,wap_balance_std,wap_balance_sum,wap_mean_mean,wap_mean_amax,wap_mean_std,wap_mean_sum,price_spread_mean,price_spread_amax,price_spread_std,price_spread_sum,bid_spread_mean,bid_spread_amax,bid_spread_std,bid_spread_sum,ask_spread_mean,ask_spread_amax,ask_spread_std,ask_spread_sum,total_volume_mean,total_volume_amax,total_volume_std,total_volume_sum,volume_imbalance_mean,volume_imbalance_amax,volume_imbalance_std,volume_imbalance_sum,bas_mean,bas_amax,bas_std,bas_sum,h_spread_l1_mean,h_spread_l1_amax,h_spread_l1_std,h_spread_l1_sum,h_spread_l2_mean,h_spread_l2_amax,h_spread_l2_std,h_spread_l2_sum,wap1_mean_450,wap1_amax_450,wap1_std_450,wap2_mean_450,wap2_amax_450,wap2_std_450,log_return1_realized_volatility_450,log_return1_sum_450,log_return2_realized_volatility_450,log_return2_sum_450,log_return_bid_price1_realized_volatility_450,log_return_bid_price1_sum_450,log_return_ask_price1_realized_volatility_450,log_return_ask_price1_sum_450,log_return_bid_size1_realized_volatility_450,log_return_bid_size1_sum_450,log_return_ask_size1_realized_volatility_450,log_return_ask_size1_sum_450,log_ask_1_div_bid_1_realized_volatility_450,log_ask_1_div_bid_1_sum_450,log_ask_1_div_bid_1_size_realized_volatility_450,log_ask_1_div_bid_1_size_sum_450,log_return_bid_price2_realized_volatility_450,log_return_bid_price2_sum_450,log_return_ask_price2_realized_volatility_450,log_return_ask_price2_sum_450,log_return_bid_size2_realized_volatility_450,log_return_bid_size2_sum_450,log_return_ask_size2_realized_volatility_450,log_return_ask_size2_sum_450,log_ask_2_div_bid_2_realized_volatility_450,log_ask_2_div_bid_2_sum_450,log_ask_2_div_bid_2_size_realized_volatility_450,log_ask_2_div_bid_2_size_sum_450,wap_balance_mean_450,wap_balance_amax_450,wap_balance_std_450,wap_balance_sum_450,wap_mean_mean_450,wap_mean_amax_450,wap_mean_std_450,wap_mean_sum_450,price_spread_mean_450,price_spread_amax_450,price_spread_std_450,price_spread_sum_450,bid_spread_mean_450,bid_spread_amax_450,bid_spread_std_450,bid_spread_sum_450,ask_spread_mean_450,ask_spread_amax_450,ask_spread_std_450,ask_spread_sum_450,total_volume_mean_450,total_volume_amax_450,total_volume_std_450,total_volume_sum_450,volume_imbalance_mean_450,volume_imbalance_amax_450,volume_imbalance_std_450,volume_imbalance_sum_450,bas_mean_450,bas_amax_450,bas_std_450,bas_sum_450,h_spread_l1_mean_450,h_spread_l1_amax_450,h_spread_l1_std_450,h_spread_l1_sum_450,h_spread_l2_mean_450,h_spread_l2_amax_450,h_spread_l2_std_450,h_spread_l2_sum_450,wap1_mean_300,...,wap_balance_std_150,wap_balance_sum_150,wap_mean_mean_150,wap_mean_amax_150,wap_mean_std_150,wap_mean_sum_150,price_spread_mean_150,price_spread_amax_150,price_spread_std_150,price_spread_sum_150,bid_spread_mean_150,bid_spread_amax_150,bid_spread_std_150,bid_spread_sum_150,ask_spread_mean_150,ask_spread_amax_150,ask_spread_std_150,ask_spread_sum_150,total_volume_mean_150,total_volume_amax_150,total_volume_std_150,total_volume_sum_150,volume_imbalance_mean_150,volume_i

In [16]:
x_val.shape

(42893, 409)

In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import os
import glob
from tqdm import tqdm
from joblib import Parallel, delayed
import gc

from sklearn.model_selection import train_test_split, KFold

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import optuna
from optuna.samplers import TPESampler

In [18]:
def objective(trial):
    
    def rmspe(y_true, y_pred):
        return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))
    
    valid = [(x_val, y_val)]
    
    param = {
        "device": "gpu",
        'boosting': 'gbdt',
        "metric": "rmse",
        "verbosity": -1,
        'learning_rate':trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        "max_depth": -1,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 1000),
        "n_estimators": trial.suggest_int("n_estimators", 100, 4000),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 100, 2000),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        'feature_fraction_bynode': trial.suggest_float("feature_fraction_bynode", 0.4, 1.0),
        'min_sum_hessian_in_leaf':trial.suggest_int("min_sum_hessian_in_leaf", 10, 100),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        'seed':1,
        'feature_fraction_seed': 1,
        'bagging_seed': 1,
        'drop_seed': 1,
        'data_random_seed': 1}

    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")
    model = LGBMRegressor(**param)
    
    model.fit(x_train, y_train, eval_set=valid, verbose=False, callbacks=[pruning_callback], early_stopping_rounds=100)

    preds = model.predict(x_val)
    
    rmspe = rmspe(y_val, preds)
    return rmspe

In [19]:
study = optuna.create_study(sampler=TPESampler(), direction='minimize', pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study.optimize(objective, n_trials=100, gc_after_trial=True)

[I 2021-08-25 01:20:02,612] A new study created in memory with name: no-name-1187a560-6436-47bd-9246-e24ae47ad29c


[LightGBM] [Warning] lambda_l2 is set=0.0001064981514788677, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.0001064981514788677
[LightGBM] [Warning] lambda_l1 is set=0.057791415235657055, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.057791415235657055
[LightGBM] [Warning] bagging_fraction is set=0.5462673950736119, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5462673950736119
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.5983359065460901, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5983359065460901
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=49, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=49
[LightGBM] [Warning] min_data_in_leaf is set=410, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=410
[LightGBM] [Warning] bagging_freq is set=1, subsampl

[I 2021-08-25 01:21:17,787] Trial 0 finished with value: 0.24922532256227206 and parameters: {'learning_rate': 0.3370694457912124, 'lambda_l1': 0.057791415235657055, 'lambda_l2': 0.0001064981514788677, 'num_leaves': 908, 'n_estimators': 1965, 'min_data_in_leaf': 410, 'feature_fraction': 0.5983359065460901, 'feature_fraction_bynode': 0.6583153230415957, 'min_sum_hessian_in_leaf': 49, 'bagging_fraction': 0.5462673950736119, 'bagging_freq': 1}. Best is trial 0 with value: 0.24922532256227206.


[LightGBM] [Warning] lambda_l2 is set=0.00018335926502107195, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00018335926502107195
[LightGBM] [Warning] lambda_l1 is set=1.3403483541368712e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.3403483541368712e-08
[LightGBM] [Warning] bagging_fraction is set=0.6783216538385552, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6783216538385552
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.9330955525426714, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9330955525426714
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=10, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=10
[LightGBM] [Warning] min_data_in_leaf is set=1529, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1529
[LightGBM] [Warning] bagging_freq is set=3, 

[I 2021-08-25 01:23:00,647] Trial 1 finished with value: 0.24636881081474052 and parameters: {'learning_rate': 0.18174690421424808, 'lambda_l1': 1.3403483541368712e-08, 'lambda_l2': 0.00018335926502107195, 'num_leaves': 42, 'n_estimators': 2574, 'min_data_in_leaf': 1529, 'feature_fraction': 0.9330955525426714, 'feature_fraction_bynode': 0.46215766132983804, 'min_sum_hessian_in_leaf': 10, 'bagging_fraction': 0.6783216538385552, 'bagging_freq': 3}. Best is trial 1 with value: 0.24636881081474052.


...

[I 2021-08-25 01:37:52,845] Trial 5 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 01:38:17,964] Trial 6 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 01:38:43,247] Trial 7 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 01:39:08,987] Trial 8 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 01:39:34,308] Trial 9 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 01:40:00,648] Trial 10 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 01:40:25,990] Trial 11 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 01:40:52,992] Trial 12 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 01:41:18,577] Trial 13 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 01:41:44,932] Trial 14 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 01:42:11,767] Trial 15 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 01:42:37,772] Trial 16 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 01:43:04,565] Trial 17 pruned. Trial was pruned at iter

[LightGBM] [Warning] lambda_l2 is set=3.628410123908276e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.628410123908276e-05
[LightGBM] [Warning] lambda_l1 is set=0.13820639380774133, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.13820639380774133
[LightGBM] [Warning] bagging_fraction is set=0.5141192661862902, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5141192661862902
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.706082456708135, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.706082456708135
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=66, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=66
[LightGBM] [Warning] min_data_in_leaf is set=616, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=616
[LightGBM] [Warning] bagging_freq is set=1, subsample_fr

[I 2021-08-25 02:16:44,359] Trial 74 pruned. Trial was pruned at iteration 257.
[I 2021-08-25 02:17:11,463] Trial 75 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 02:18:15,350] Trial 76 pruned. Trial was pruned at iteration 100.
[I 2021-08-25 02:18:41,861] Trial 77 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 02:19:09,236] Trial 78 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 02:19:35,605] Trial 79 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 02:20:03,202] Trial 80 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 02:21:02,982] Trial 81 finished with value: 0.24978658658395092 and parameters: {'learning_rate': 0.353291122207088, 'lambda_l1': 0.04054325771950118, 'lambda_l2': 0.00011729030681188265, 'num_leaves': 849, 'n_estimators': 2742, 'min_data_in_leaf': 801, 'feature_fraction': 0.6906301362326259, 'feature_fraction_bynode': 0.7618040074395416, 'min_sum_hessian_in_leaf': 62, 'bagging_fraction': 0.4896150596458005, 'bagging_freq': 1}. Best

[LightGBM] [Warning] lambda_l2 is set=0.00010222711683530481, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00010222711683530481
[LightGBM] [Warning] lambda_l1 is set=0.039279876582080764, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.039279876582080764
[LightGBM] [Warning] bagging_fraction is set=0.509586203208689, subsample=1.0 will be ignored. Current value: bagging_fraction=0.509586203208689
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6996433196700522, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6996433196700522
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=61, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=61
[LightGBM] [Warning] min_data_in_leaf is set=826, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=826
[LightGBM] [Warning] bagging_freq is set=1, subsampl

[I 2021-08-25 02:21:32,885] Trial 82 pruned. Trial was pruned at iteration 22.
[I 2021-08-25 02:21:59,048] Trial 83 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 02:22:26,889] Trial 84 pruned. Trial was pruned at iteration 13.
[I 2021-08-25 02:22:55,125] Trial 85 pruned. Trial was pruned at iteration 9.
[I 2021-08-25 02:23:21,776] Trial 86 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 02:23:49,086] Trial 87 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 02:24:15,492] Trial 88 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 02:25:11,031] Trial 89 pruned. Trial was pruned at iteration 83.
[I 2021-08-25 02:25:38,509] Trial 90 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 02:26:05,070] Trial 91 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 02:26:37,803] Trial 92 pruned. Trial was pruned at iteration 29.
[I 2021-08-25 02:27:04,443] Trial 93 pruned. Trial was pruned at iteration 5.
[I 2021-08-25 02:27:31,135] Trial 94 pruned. Trial was prune

In [20]:
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

Number of finished trials: 100
Best trial: {'learning_rate': 0.011413959842017922, 'lambda_l1': 6.393484937256699e-07, 'lambda_l2': 9.688957258498976e-06, 'num_leaves': 815, 'n_estimators': 3110, 'min_data_in_leaf': 843, 'feature_fraction': 0.4870690801684412, 'feature_fraction_bynode': 0.7646999898853415, 'min_sum_hessian_in_leaf': 46, 'bagging_fraction': 0.4876475698163599, 'bagging_freq': 6}


In [21]:
optuna.visualization.plot_optimization_history(study)